<a href="https://colab.research.google.com/github/napstruck/DetectTheAudio/blob/branch-1/Copy_of_Major_Project_FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation and Data set extraction

In [2]:
!pip install --upgrade tensorflow-federated librosa

import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import librosa
import os
import zipfile
from google.colab import drive
from tensorflow.keras.layers import Input, Concatenate, Dense, Flatten
from tensorflow.keras.models import Model

ERROR:jax._src.xla_bridge:Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/xla_bridge.py", line 438, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.11/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 85, in initialize
    options = xla_client.generate_pjrt_gpu_plugin_options()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: module 'jaxlib.xla_client' has no attribute 'generate_pjrt_gpu_plugin_options'


In [3]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import librosa
import os
from tensorflow.keras.layers import Input, Concatenate, Dense, Flatten


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import zipfile
import os

# Path to the zip file
zip_path = "/content/drive/MyDrive/audio_dataset.zip"

# Directory to extract the dataset
extract_path = "/content/dataset"

# Create the extraction directory
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")

Dataset extracted successfully!


In [6]:
dataset_path = os.path.join(extract_path, "audio_dataset")

In [7]:
def extract_features(audio, sample_rate=16000):
    audio = audio.numpy()
    features = {
        'mfccs': librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13).T,
        'chroma': librosa.feature.chroma_stft(y=audio, sr=sample_rate).T,
        'spectral_contrast': librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T,
        'zero_crossing_rate': librosa.feature.zero_crossing_rate(audio).T,
        'rms_energy': librosa.feature.rms(y=audio).T,
        'mel_spectrogram': librosa.feature.melspectrogram(y=audio, sr=sample_rate).T
    }
    return features

In [8]:
def preprocess_audio(file_path, label):
    audio = tf.io.read_file(file_path)
    audio, _ = tf.audio.decode_wav(audio, desired_channels=1)
    audio = tf.squeeze(audio, axis=-1)
    features = tf.py_function(extract_features, [audio],
                              Tout=[tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32])
    return { 'mfccs': features[0], 'chroma': features[1], 'spectral_contrast': features[2],
             'zero_crossing_rate': features[3], 'rms_energy': features[4], 'mel_spectrogram': features[5] }, label


In [9]:
fake_files = tf.data.Dataset.list_files(os.path.join(dataset_path, "fake/*.wav"), shuffle=False)
real_files = tf.data.Dataset.list_files(os.path.join(dataset_path, "real/*.wav"), shuffle=False)


In [10]:
fake_dataset = fake_files.map(lambda x: (x, 0))  # 0 for fake
real_dataset = real_files.map(lambda x: (x, 1))  # 1 for real

In [11]:
full_dataset = fake_dataset.concatenate(real_dataset).map(preprocess_audio)
full_dataset = full_dataset.shuffle(buffer_size=88284, seed=42)

In [12]:
dataset_size = 88284
chunk_size = dataset_size // 6
chunks = [full_dataset.skip(i * chunk_size).take(chunk_size) for i in range(6)]
train_clients_data = chunks[:5]
test_data = chunks[5]


In [13]:
train_clients_data = chunks[:5]
test_data = chunks[5]


In [14]:
def create_federated_model():
    mfcc_input = Input(shape=(100, 13), name='mfccs')
    chroma_input = Input(shape=(100, 12), name='chroma')
    spectral_contrast_input = Input(shape=(100, 7), name='spectral_contrast')
    zero_crossing_input = Input(shape=(100, 1), name='zero_crossing_rate')
    rms_input = Input(shape=(100, 1), name='rms_energy')
    mel_input = Input(shape=(100, 128), name='mel_spectrogram')

    flattened_features = [Flatten()(mfcc_input), Flatten()(chroma_input), Flatten()(spectral_contrast_input),
                          Flatten()(zero_crossing_input), Flatten()(rms_input), Flatten()(mel_input)]
    concatenated = Concatenate()(flattened_features)
    x = Dense(128, activation='relu')(concatenated)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)

    keras_model = Model(inputs=[mfcc_input, chroma_input, spectral_contrast_input,
                                zero_crossing_input, rms_input, mel_input], outputs=output)

    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=train_clients_data[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

In [15]:
!pip install --upgrade tensorflow-federated
import tensorflow as tf
import tensorflow_federated as tff


In [16]:
!pip install --upgrade pip setuptools wheel

In [20]:
def client_optimizer_fn():
    return lambda: tff.learning.optimizers.build_sgdm(learning_rate=0.001)
def server_optimizer_fn():
    return lambda: tff.learning.optimizers.build_sgdm(learning_rate=0.001)

In [21]:
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=create_federated_model,
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

AttributeError: 'function' object has no attribute 'initialize'

In [ ]:
state = iterative_process.initialize()
for round_num in range(10):
    state, metrics = iterative_process.next(state, train_clients_data)
    print(f"Round {round_num + 1}, Metrics: {metrics}")

In [87]:
# mfcc_input = Input(shape=(100, 13), name='mfccs') # Assuming max 100 time steps for MFCCs
# chroma_input = Input(shape=(100, 12), name='chroma') # Assuming max 100 time steps for Chroma
# spectral_contrast_input = Input(shape=(100, 7), name='spectral_contrast') # Assuming max 100 time steps for Spectral Contrast
# zero_crossing_input = Input(shape=(100, 1), name='zero_crossing_rate') # Assuming max 100 time steps for Zero-Crossing Rate
# rms_input = Input(shape=(100, 1), name='rms_energy') # Assuming max 100 time steps for RMS Energy
# mel_input = Input(shape=(100, 128), name='mel_spectrogram') # Assuming max 100 time steps for Mel Spectrogram

In [88]:
# flattened_features = [
#     Flatten()(mfcc_input),
#     Flatten()(chroma_input),
#     # Flatten()(spectral_contrast_input),
#     Flatten()(zero_crossing_input),
#     Flatten()(rms_input),
#     Flatten()(mel_input)
# ]
# concatenated = Concatenate()(flattened_features)

In [89]:
# x = Dense(128, activation='relu')(concatenated)
# x = Dense(64, activation='relu')(x)
# output = Dense(1, activation='sigmoid')(x)

In [90]:
# model = tf.keras.Model(
#     inputs=[mfcc_input, chroma_input, spectral_contrast_input, zero_crossing_input, rms_input, mel_input],
#     outputs=output
# )

In [91]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [92]:
def create_federated_model():
    # Define the model structure
    mfcc_input = Input(shape=(100, 13), name='mfccs')  # Assuming max 100 time steps for MFCCs
    chroma_input = Input(shape=(100, 12), name='chroma')  # Assuming max 100 time steps for Chroma
    spectral_contrast_input = Input(shape=(100, 7), name='spectral_contrast')  # Assuming max 100 time steps for Spectral Contrast
    zero_crossing_input = Input(shape=(100, 1), name='zero_crossing_rate')  # Assuming max 100 time steps for Zero-Crossing Rate
    rms_input = Input(shape=(100, 1), name='rms_energy')  # Assuming max 100 time steps for RMS Energy
    mel_input = Input(shape=(100, 128), name='mel_spectrogram')  # Assuming max 100 time steps for Mel Spectrogram

    flattened_features = [
        Flatten()(mfcc_input),
        Flatten()(chroma_input),
        Flatten()(spectral_contrast_input),
        Flatten()(zero_crossing_input),
        Flatten()(rms_input),
        Flatten()(mel_input)
    ]

    concatenated = Concatenate()(flattened_features)
    x = Dense(128, activation='relu')(concatenated)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(
        inputs=[mfcc_input, chroma_input, spectral_contrast_input,
                zero_crossing_input, rms_input, mel_input],
        outputs=output
    )

    # Do NOT compile the model before passing it to TFF
    return tff.learning.models.from_keras_model(
        model,
        input_spec=train_clients_data[0].element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

In [96]:
!pip install --upgrade tensorflow-federated
import tensorflow as tf
import tensorflow_federated as tff


In [100]:
def client_optimizer_fn():
    return tf.keras.optimizers.Adam(learning_rate=0.001)
def server_optimizer_fn():
    return tf.keras.optimizers.Adam(learning_rate=0.001)
# Build the federated averaging process
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    create_federated_model,
    client_optimizer_fn=client_optimizer_fn,  # Pass the function here
    server_optimizer_fn=server_optimizer_fn   # Pass the function here
)

AttributeError: 'function' object has no attribute 'initialize'